# Welcome to the Well Data Interactive Visualization Tutorial for Transform 2020
## By: Ashley Russell, Product Owner@Equinor

## Agenda:


*   The soft stuff - why make pretty pictures that have interaction? (15 min)
*   Getting the data set together for visualizating and interacting (30 min)
*   Building blocks (45 min)
  * First with one well
  * Then with more wells
*   Your first python dashboard (30 min)






## 1. The Soft Stuff.  

You have been working on a coding project and are asked to present "in the raw" to demonstrate what you have been doing.  What do you show, how do you show it and why?

## Color
Instant recognitition even before you say anything.
<p><img src="https://thumbor.forbes.com/thumbor/960x0/https%3A%2F%2Fblogs-images.forbes.com%2Fevamurray%2Ffiles%2F2019%2F03%2FGerman-discipline.jpg" ></p>

In geology and geophysics we are atuned to certain colors representing domain-specific things.

What would this mean to you?

<p><img src="https://www.color-hex.com/palettes/43054.png" ></p>

And this one?

<p><img src="https://i.stack.imgur.com/E4Cpw.png" ></p>

#### (If you can tell me which flag this is...bonus points)

Key point is that colors convey silent messages, choose wisely and know your audience.  This becomes important when we go to the next soft subject.

More reading found here: https://agilescientific.com/blog/2013/8/20/five-more-things-about-colour.html

## Familiarity

Most geoscientists, especially those working in energy/mining companies are familiar with specific domain software.  These software have existed in these industries for the past 25 years and they have left a significant imprint on how we expect geologic data to be presented.  Think about how many powerpoints you may have seen that have screenshots from certain petrophysical or oil and gas software in them.  

As we try to move to more raw code sharing and embedded visualization, remember the bias that many in your audience may have on how they expect data to be presented due to their work with conventional software.  And of course, basic geological logic and expectations...

Some things to think about:

* What units are you in?  
* What scale to use?  
* What is familiar to your audience? Are you presenting to hardcore petrophysicists or more general geoscientists?  Geophysicists? 
* What visualization dimensions (ie length/width) make sense?
* How can you ensure that the audience is seeing things in the right units?

## Unlocking Storytelling with Interactivity

From my experience, adding interactivity to your visualizations within a live sharing session in a notebook (or web GUI) for demoing to "non-coding" geoscientists has two benefits:

1. Prevents "overwhelming"
2. Proves that what you are doing actually works (question-ready!)

First, the topic of being overwhelming.  If you create some huge plot or dashboard showing well data from 10 different wells with cross plots, histograms, and a well section all at one time, it may look impressive, but more than likely, especially if you have an audience that doesn't look at a lot of python notebooks, they will be very confused.  And most likely, any units or measurements will be too small to read.  Begin by showing each of the components and the interactivity available there. For example, swap between wells, or between lithologies on the individual plots.  Allow them to work with the individual components before revealing the big final dashboard or summarizing plot.  We will have some examples of this below.

Doesn't this terrify you?:
<p><img src="https://imgix.datadoghq.com/img/dashboarding/Dashboards-vid-01-v01.png" ></p>

Second, interactivity allows you to be "question-ready".  I expect that when presenting you are trying to answer a question or investigate a certain problem with your code.  Anticipate what the audience will ask about and allow them through interactivity to ask these questions on the fly, while you change the plots to show them what they are interested in.  For example, an audience member may ask "We didn't see this problem in Well X, does your data show that?" - use the interactivity you have built in to allow you to quickly change the plot to look at Well X.  Your audience will trust you more and it often leads to great discussions.

# Trust

Overall, all these "soft things" above drive at gaining trust with your audience when presenting code.  At least in my workplace, demonstrating in a python notebook is still very foreign to the geoscience community.  It is up to you to prove that your work is of the same quality (perhaps better) than the software systems they currently trust!

## 2. Preparation

We don't need to worry about each of you getting packages installed or python working while we use Google Colab, but when you get back to your own environment, here are the packages and the settings to use in your Jupyter Notebook to get all the cool stuff below working.

Please select the "Open in Playground" button at the top of Google Colab

### 2.1 Packages Install

In [0]:
## But we do need to install a few packages into the Google Colab environment (most come with it already):
!pip install plotly_express
!pip install --upgrade hvplot
!pip install --upgrade bokeh
!pip install --upgrade holoviews
!pip install lasio
!pip install -U ipykernel
!pip install --upgrade panel

In [0]:
## Some common packages you already probably know:
import pandas as pd
import matplotlib.pyplot as plt
import lasio as ls
import numpy as np

## Some perhaps new packages we will use today:
import plotly_express as px
import hvplot
import hvplot.pandas
import holoviews as hv
import panel as pn

### 2.2 Packages Info

We will be working with two main visualization packages today - plotly express and Holoviews (hvplot).  Both of these packages are wrappers  around matplotlib to create amazingly easy interactive visualizations.  Then we will be using a dashboarding package called panel to combine our plots together.

* https://plotly.com/python/plotly-express/ - I highly recommend reading through the Reference Documentation on this page.
* https://hvplot.holoviz.org/ - bit more complex than plotly-express
* https://panel.holoviz.org/ - stitching plots together

### 2.3 Single Well File Dataset Load

We will be using open source data from Australia today - https://nopims.dmp.wa.gov.au/Nopims/

You should be accessing the following file (you will need Google Drive): https://drive.google.com/file/d/1Lws7PztcgZlKT4TQNfmojDesKSOIZ56X/view?usp=sharing

In [0]:
## Then we need to load some data (we will use a .las file for now from a well in the NOPIMS Australia dataset)
from google.colab import drive
drive.mount('/content/drive/')

In [0]:
filepath = r'/content/drive/My Drive/Pharos-1 Ascii Drill Data_481.5-5220m.LAS'

In [0]:
def load_las_to_df(filepath):
  las = ls.read(filepath)
  wellname = las.well.WELL.value
  df=las.df()
  df.reset_index(inplace=True)
  df['DEPTH'] = df['UNKNOWN']
  df['WELLNAME'] = wellname
  df.drop(axis=1, columns='UNKNOWN', inplace=True)
  return df;

In [0]:
pharos_curve_df = load_las_to_df(filepath)

In [0]:
pharos_curve_df

Now, with this single well dataset - mudlogs from a well called Pharos-1 offshore Australia, we want to illustrate some relationships between the drilling parameters and the lithology.  The lithology however is in a separate file, let's load that in and take a look.  We need to look at the data structure to find the best way to combine them - especially if we want some sort of interactivity with lithology.

You should be accessing the following file: https://drive.google.com/file/d/1splcE2E_SXjsRq4olHmlqFPHwDNtSLZO/view?usp=sharing

In [0]:
filepath = r'/content/drive/My Drive/Pharos-1 Ascii Lithology Intrepretation_2330m to 5220m.xlsx'

In [0]:
def load_lithxls_to_df(filepath):
  df_lith = pd.read_excel(filepath, header=1, skiprows=1)
  df_lith.dropna(axis=0, inplace=True)
  df_lith.reset_index(drop=True, inplace=True)
  df_lith['Litho_description'] = df_lith['Litho description']
  df_lith['To depth'] = df_lith['To depth'].astype('float64')
  df_lith['From depth'] = df_lith['From depth'].astype('float64')
  df_lith.drop(axis=1, columns='Litho description', inplace=True)
  return df_lith;

In [0]:
pharos_lith_df = load_lithxls_to_df(filepath)

In [0]:
pharos_lith_df

Looks like our data isn't quite as easy to merge as we like.  The lithology is expressed as ranges of depth.  Luckily, pandas can sort this out for us using IntervalIndex to build a list of intervals.  In order to get interactivity on a variable like this, we need to have a value filled in for each row in our mud log dataframe so we have a column called "Lithology".

In [0]:
pharos_lith_df.index = pd.IntervalIndex.from_arrays(pharos_lith_df['From depth'],pharos_lith_df['To depth'],closed='both')
pharos_curve_df['LITHOLOGY'] = pharos_curve_df['DEPTH'].apply(
    lambda x : pharos_lith_df.iloc[pharos_lith_df.index.get_loc(x)]['Litho_description'])

In [0]:
pharos_lith_df

In [0]:
## There are a few errors in the file where the matching didn't work.  
## The original file would need to be cleaned up.  
## For simplicity, we will drop those rows
def drop_weird_results(dataframe, columnname, searchstring):
  dataframe[columnname]=dataframe[columnname].astype('str')
  dataframe = dataframe[~dataframe[columnname].str.contains(searchstring)]
  dataframe.reset_index(drop=True, inplace=True)
  return dataframe;

In [0]:
pharos_curve_df = drop_weird_results(pharos_curve_df, 'LITHOLOGY', 'Name:')

Now we have a dataset for a single well with a category variable we can use for interactivity later...

In [0]:
pharos_curve_df.tail()

## 3. Building Blocks

### 3.1 Single well visualization with Pharos-1

#### 3.1a A first look at plotly express

Let's start with a simple implementation of comparing our drilling depth with the pipe pressure.  We will add the temperature of the mud coming out of hole as our color.  This plot should give us some idea about the temperatures and pressures that occured during drilling this well.

In [0]:
px.scatter(pharos_curve_df, x='DEPTH', y='STAND PIPE PRESSURE', color='TEMPERATURE OUT')

You can see with a plotly express plot, you get some neat tools for examining your plot - you can hover over data points, read values, and zoom in and out.

#### Question 3.1a1:

How could we make this plot easier for a geologist, driller, or petrophysicist to look at?

In [0]:
px.scatter(pharos_curve_df, x='STAND PIPE PRESSURE', y='DEPTH', color='TEMPERATURE OUT', 
           labels={'STAND PIPE PRESSURE':'STAND PIPE PRESSURE (PSI)', 'DEPTH':'DEPTH (m)', 'TEMPERATURE OUT':'TEMPERATURE OUT (degC)'}, 
           range_y=[5220,0], range_color=[0,50], width=700, height=1000)

It is important to point out that one of the things that irritates me most when others plot well data is the lack of reversing the depth range!  But, as you will see, the parameters to reverse the y-axis differ from plotting package to plotting package.  Take another look at how plotly express does it above.

#### Question 3.1a2:

What would happen if we wanted to swap out the coloring for a string variable?  We are after all interested in looking at our drilling parameters based on lithology.  This well has both carbonate and clastic sections, can we see if an relationship exists between lithology our mudlogging pressure and temperature measurements?

In [0]:
px.scatter(pharos_curve_df, x='STAND PIPE PRESSURE', y='DEPTH', color='LITHOLOGY', 
           labels={'STAND PIPE PRESSURE':'STAND PIPE PRESSURE (PSI)', 'DEPTH':'DEPTH (m)', 'TEMPERATURE OUT':'TEMPERATURE OUT (degC)'}, 
           range_y=[5220,2290], range_color=[0,50], width=700, height=1000)

#### Question 3.1a3

Does this lithology plot look right to you?

#### 3.1b Making a color map and category map

We can do two things to improve the lithology rendering on this plot.  The first is to make a color map to assign certain colors to the different categories.  The second is to map lithologies to larger categories to make it simpler to read (we will continue to explore this later)

In [0]:
lithology_list = list(pharos_curve_df.LITHOLOGY.unique())

In [0]:
lithology_list

In [0]:
detailed_color_list = ['#adadad', '#adadad', '#41a4fa', '#aad5fa', '#33e2f2', '#a0b8ba', '#faf09b', '#46eefa', '#aad5fa',
              '#b0b7f5', '#faf09b', '#46eefa', '#b4debe', '#faf09b', '#b4debe', '#cbf5ae', '#faf09b', '#b4debe', 
              '#b4debe', '#b4debe']  ## Adapted from USGS Lithology colors

In [0]:
dict_detailed_colors = dict(zip(lithology_list, detailed_color_list))

In [0]:
dict_detailed_colors

In [0]:
category_list = ['Undefined', 'Undefined',  'Carbonate', 'Carbonate', 'Mixed Carbonate/Clastic', 'Chemical', 'Clastic',
                 'Carbonate', 'Carbonate', 'Chemical', 'Clastic', 'Carbonate', 'Clastic', 'Clastic', 'Clastic', 'Clastic',
                 'Clastic', 'Clastic', 'Clastic', 'Clastic']

In [0]:
category_colors = {'Undefined':'#adadad', 'Carbonate':'#6efaff', 'Mixed Carbonate/Clastic':'#4ce0c8',
                   'Chemical':'#cabae3', 'Clastic':'#faf09b', 'Volcanic':'#f3c1f5'}

And now to build the dictionaries we need and add the lithology categories to our dataframe (we will use this later)

In [0]:
lith_category = dict(zip(lithology_list, category_list))
pharos_curve_df.loc[:,'LITH_CATEGORY'] = pharos_curve_df['LITHOLOGY'].map(lith_category)

In [0]:
pharos_curve_df

Now we can generate two new plots with these mappings:

In [0]:
px.scatter(pharos_curve_df, x='STAND PIPE PRESSURE', y='DEPTH', color='LITHOLOGY',
           labels={'STAND PIPE PRESSURE':'STAND PIPE PRESSURE (PSI)', 'DEPTH':'DEPTH (m)', 'TEMPERATURE OUT':'TEMPERATURE OUT (degC)'}, 
           color_discrete_map=dict_detailed_colors,
           range_y=[5220,2290], range_color=[0,50], width=700, height=1000)

In [0]:
px.scatter(pharos_curve_df, x='STAND PIPE PRESSURE', y='DEPTH', color='LITH_CATEGORY', 
           labels={'STAND PIPE PRESSURE':'STAND PIPE PRESSURE (PSI)', 'DEPTH':'DEPTH (m)', 'TEMPERATURE OUT':'TEMPERATURE OUT (degC)'}, 
           color_discrete_map=category_colors, range_y=[5500,0], width=800, height=1000)

#### 3.1c  Lines versus Scatter?
We've been displaying this data as points, but typically for well logs, we view them as lines.  Let's see what this looks like.  

In [0]:
px.line(pharos_curve_df, x='STAND PIPE PRESSURE', y='DEPTH', hover_name='TEMPERATURE OUT', 
           labels={'STAND PIPE PRESSURE':'STAND PIPE PRESSURE (PSI)', 'DEPTH':'DEPTH (m)', 'TEMPERATURE OUT':'TEMPERATURE OUT (degC)'}, 
           range_y=[5220,0], width=600, height=1000)

Looks like a well log right!  Unfortunately, plotly express doesn't support coloring the line with different lithologies..so we need to move to another package - Holoviews and its wrapper for pandas: hvplot.  We are leaving plotly express for now, but there is way more functionality and you can even dig deeper into plotly itself.

#### 3.1d A first look at hvplot

Plotly_express is great for quick, single plots with interactive investigation.  Now, we will get introduced to HoloViews - hvplot, which allows us to do some more complicated things using the bokeh plotting engine.  Most importantly, we can begin adding easily adding drop-down menus to build interactivity through selection.  Google Colab doesn't fully support holoviews, so we have to load the extension for each plot (you won't have to do this with a regular Jupyter notebook).

In [0]:
hv.extension('bokeh')

pharos_curve_df.hvplot(x='DEPTH', y='STAND PIPE PRESSURE', invert=True, flip_yaxis=True, height=1000, width=700).opts(fontsize={'labels': 16,'xticks': 14, 'yticks': 14}, 
                                                                                                                      xlabel='DEPTH (m)', ylabel='STAND PIPE PRESSURE (PSI)')

#### Question 3.1d1
What are the major differences in how plots are formatted in hvplot versus plotly express?

####3.1e Combining plots in hvplot
We can combine plots together super easily in hvplot.  We will use this later to build a well section!  For now, our plots are static, and we can show a curve and a table of that curve side by side.  The tables in hvplot are also interactive - You can very quickly sort.

In [0]:
hv.extension('bokeh')
curve = pharos_curve_df.hvplot(x='DEPTH', y='STAND PIPE PRESSURE', invert=True, flip_yaxis=True, 
                       height=800, width=600).opts(fontsize={'labels': 16,'xticks': 14, 'yticks': 14}, 
                       xlabel='DEPTH (m)', ylabel='STAND PIPE PRESSURE (PSI)')
table = pharos_curve_df.hvplot.table(columns=['DEPTH','STAND PIPE PRESSURE'])

curve + table

####3.1f "easy" well section with hvplot subplots
There is a cheater method for building a well section in hvplot - however without extensive customization, it is only good for well logs of a single type (ie, linear...). 

In [0]:
def getnumeric(df, depthcurvename):
  curve_list=list(df.columns[(df.dtypes.values == np.dtype('float64'))])
  curve_list.remove(depthcurvename)
  return curve_list;

In [0]:
curve_list = getnumeric(pharos_curve_df, 'DEPTH')

In [0]:
len(curve_list)

In [0]:
hv.extension('bokeh')
pharos_curve_df.hvplot(x='DEPTH', y=curve_list, subplots=True, invert=True, flip_yaxis=True, shared_axes=False, width=300, height=800).cols(len(curve_list))

####Question 3.1f1
What could make this well section better?  

Units, synchronized scrolling...

#### 3.1g A better well section
Let's recreate the hvplot well section above, but now we can make it better by using the hvplot Layout list function (http://holoviews.org/user_guide/Composing_Elements.html) tools to help us when the number of curves can change from one file to another instead of using the plus sign notation.

In [0]:
hv.extension('bokeh')

def curve_plot(log, df, depthname):
  aplot = df.hvplot(x=depthname, y=log, invert=True, flip_yaxis=True, shared_axes=True,
                       height=800, width=300).opts(fontsize={'labels': 16,'xticks': 14, 'yticks': 14})
  return aplot;

plotlist = [curve_plot(x, df=pharos_curve_df, depthname='DEPTH') for x in curve_list]
well_section = hv.Layout(plotlist).cols(len(curve_list))
well_section

Now we get synchronized scrolling!  With this function, we can also now start adding customization for different types of curves.  You could think about this for log scales for resistivity, or green shading for gamma ray.  Let's do a quick change to first curve below.

In [0]:
hv.extension('bokeh')

def curve_plot(log, df, depthname):
  if log == 'ROP OFFLINE':
    aplot = df.hvplot(x=depthname, y=log, invert=True, flip_yaxis=True, shared_axes=True,
                       height=800, width=300).opts(fontsize={'labels': 16,'xticks': 14, 'yticks': 14}).opts(line_dash='dashed', color='red')
  else:
    aplot = df.hvplot(x=depthname, y=log, invert=True, flip_yaxis=True, shared_axes=True,
                       height=800, width=300).opts(fontsize={'labels': 16,'xticks': 14, 'yticks': 14})
  return aplot;

plotlist = [curve_plot(x, df=pharos_curve_df, depthname='DEPTH') for x in curve_list]
well_section = hv.Layout(plotlist).cols(len(curve_list))
well_section

#### 3.1g Adding in Panel

Panel allows us to take plots from multiple plotting packages and combine them together.  Let's re-create one of the plotly express plots and combine it with a table from hvplot.  We will use the pn.Row to organize the plots - we will just put everything in one row for now.

In [0]:
hv.extension('bokeh')
pn.extension('plotly')

lithology_pressure = px.scatter(pharos_curve_df, x='STAND PIPE PRESSURE', y='DEPTH', color='LITH_CATEGORY',  
           labels={'STAND PIPE PRESSURE':'STAND PIPE PRESSURE (PSI)', 'DEPTH':'DEPTH (m)', 'TEMPERATURE OUT':'TEMPERATURE OUT (degC)'}, 
           color_discrete_map=category_colors, range_y=[5500,0], width=800, height=1000)

curve = px.line(pharos_curve_df, x='STAND PIPE PRESSURE', y='DEPTH', 
           labels={'STAND PIPE PRESSURE':'STAND PIPE PRESSURE (PSI)', 'DEPTH':'DEPTH (m)', 'TEMPERATURE OUT':'TEMPERATURE OUT (degC)'}, 
           range_y=[5220,0], width=600, height=1000)

table = pharos_curve_df.hvplot.table(columns=['DEPTH','STAND PIPE PRESSURE', 'LITH_CATEGORY'])

pn.Row(
    pn.Row("## Major Lithologies vs Stand Pipe Pressure\nHover over the plot for more information.", curve, lithology_pressure, table)
)

The last thing we will take a look at with panel is adding some basic interactivity.  Remember the plot above with the single curve and the table - let's try making this interactive!

### 3.2 Multiple Wells
We've gone through some options for a single well, but often the power in interactive well data visualization comes from comparing multiple wells at once.

Be aware that one thing I have come across in presenting is that once you start showing well data from multiple wells, a map of where those wells located becomes important to highlight any spatial distances between well log data you are comparing.  This means be prepared to add in more "well header" data to your dataframe - importantly the latitude and longitude.

Also, now is when we need to talk a little bit more about dataframe structuring and the "by" parameter in hvplot especially.  It is important to ensure that within a single column you have the variables you want to provide the interactivity on.  Let's go through a small example with plotly express and then with hvplot below with just two wells.

First, we need to load the data for the new well Poisidon North-1 (I've cheated a little to pick two already standardized las and lithology files - standardizing the well log data across wells is a whole huge process in itself).

Then we will merge the dataframes together.

Files are here:
1. Las Mud Log: https://drive.google.com/file/d/1j1sLFl8Z31qQTfrGoDU6gHsX0yx1ShKA/view?usp=sharing
2. Lithology report: https://drive.google.com/file/d/1w4OJln6mzw-hDCe3YTRVVAsTLI5RDIcv/view?usp=sharing

#### 3.2a A little data wrangling...
We need to get both wells' data and lithologies into a single dataframe.

In [0]:
filepath = r'/content/drive/My Drive/Poseidon North-1 Ascii Drill Data_508-5287m.LAS'

In [0]:
poseidon_curve_df = load_las_to_df(filepath)

In [0]:
filepath = r'/content/drive/My Drive/Poseidon North-1 Ascii Lithology Intrepretation_2459m to 5287m.xlsx'

In [0]:
poseidon_lith_df = load_lithxls_to_df(filepath)

In [0]:
poseidon_lith_df.index = pd.IntervalIndex.from_arrays(poseidon_lith_df['From depth'],poseidon_lith_df['To depth'],closed='both')
poseidon_curve_df['LITHOLOGY'] = poseidon_curve_df['DEPTH'].apply(
    lambda x : poseidon_lith_df.iloc[poseidon_lith_df.index.get_loc(x)]['Litho_description'])
poseidon_curve_df = drop_weird_results(poseidon_curve_df, 'LITHOLOGY', 'Name:')

And we need to update our color mapping as Poseidon North has some new lithologies we haven't seen before!

In [0]:
missing = sorted(list(set(lithology_list) ^ set(list(poseidon_curve_df.LITHOLOGY.unique()))))

In [0]:
missing

In [0]:
dict_detailed_colors.update(dict(zip(missing, ['#faf09b', '#46eefa', '#b4debe', '#cbf5ae', '#b4debe', '#faf09b', '#adadad', '#cbf5ae', '#f3c1f5'])))
lith_category.update(dict(zip(missing, ['Clastic', 'Chemical', 'Clastic', 'Clastic', 'Clastic', 'Clastic', 'Undefined', 'Clastic', 'Volcanic'])))
poseidon_curve_df.loc[:,'LITH_CATEGORY'] = poseidon_curve_df['LITHOLOGY'].map(lith_category)

In [0]:
dict_detailed_colors

We need to combine our two well dataframes into one.  First, we will add the WGS84 coordinates from the well header to each well's dataframe.  Remember we added the wellname as it's own column?  This is so we can keep track of the records that come from each well and this will become important for selections.

In [0]:
poseidon_curve_df['LATITUDE'] = -13.573562
pharos_curve_df['LATITUDE'] = -13.681989
poseidon_curve_df['LONGITUDE'] = 122.340154
pharos_curve_df['LONGITUDE'] = 122.399306

In [0]:
multiple_wells_df = pd.concat([poseidon_curve_df, pharos_curve_df], ignore_index=True)

In [0]:
multiple_wells_df

In [0]:
multiple_wells_df.to_csv("/content/drive/My Drive/mutliple_wells_df.csv")

#### 3.2b The by keyword in hvplot

It is very quick and easy now to categorize on the wellname series.  In hvplot, we can overlay the two wells using the by parameter. Note that this differs from groupby which I will demo soon.

In [0]:
hv.extension('bokeh')
multiple_wells_df.hvplot(x='DEPTH', y='STAND PIPE PRESSURE', by='WELLNAME', invert=True, flip_yaxis=True, height=800, width=600).opts(fontsize={'labels': 16,'xticks': 14, 'yticks': 14}, 
                                                                                                                                      xlabel='DEPTH (m)', ylabel='STAND PIPE PRESSURE (PSI)')

In [0]:
hv.extension('bokeh')

curve_list = getnumeric(multiple_wells_df, 'DEPTH')

def curve_plot(log, df, depthname):
  aplot = df.hvplot(x=depthname, y=log, by='WELLNAME', invert=True, flip_yaxis=True, shared_axes=True, alpha=0.4,
                       height=800, width=300).opts(fontsize={'labels': 12,'xticks': 10, 'yticks': 10}).opts(legend_position='top')
  return aplot;

plotlist = [curve_plot(x, df=multiple_wells_df, depthname='DEPTH') for x in curve_list]
well_section = hv.Layout(plotlist).cols(len(curve_list))
well_section

#### 3.2c A little bit more on plotly express
There are two easy options in plotly express for dealing with multiple wells - the first is the facet col option.  The second is the simple color option.

In [0]:
hv.extension('bokeh')
pn.extension('plotly')

lithology_pressure = px.scatter(multiple_wells_df, x='STAND PIPE PRESSURE', y='DEPTH', facet_col="WELLNAME", color='LITH_CATEGORY', 
           labels={'STAND PIPE PRESSURE':'STAND PIPE PRESSURE (PSI)', 'DEPTH':'DEPTH (m)', 'TEMPERATURE OUT':'TEMPERATURE OUT (degC)'}, 
           color_discrete_map=category_colors, range_y=[5500,0], width=800, height=1000)

curve = px.line(multiple_wells_df, x='STAND PIPE PRESSURE', y='DEPTH', color="WELLNAME", hover_name='WELLNAME', 
           labels={'STAND PIPE PRESSURE':'STAND PIPE PRESSURE (PSI)', 'DEPTH':'DEPTH (m)', 'TEMPERATURE OUT':'TEMPERATURE OUT (degC)'}, 
           range_y=[5500,0], width=800, height=1000)

temp = px.scatter(multiple_wells_df, x='STAND PIPE PRESSURE', y='DEPTH', facet_col="WELLNAME", color='TEMPERATURE OUT', 
           labels={'STAND PIPE PRESSURE':'STAND PIPE PRESSURE (PSI)', 'DEPTH':'DEPTH (m)', 'TEMPERATURE OUT':'TEMPERATURE OUT (degC)'}, 
           range_y=[5500,0], range_color=[0,50], width=800, height=1000)

table = multiple_wells_df.hvplot.table(columns=['WELLNAME', 'DEPTH', 'STAND PIPE PRESSURE', 'TEMPERATURE OUT', 'LITH_CATEGORY'])

map = px.scatter_geo(multiple_wells_df, lat="LATITUDE", lon="LONGITUDE", color="WELLNAME", scope='world', width=800, height=600)
map.update_layout(showlegend=False)

pn.Row(
    pn.Column("## Major Lithologies vs Stand Pipe Pressure and Temperature Out\nHover over the plot for more information.", table, map),
    pn.Row(curve, temp, lithology_pressure)
)

BUT!  There isn't any cross interactivity here - what if we want to do that?

## Dashboarding with Selection

Now we will move outside of Google Colab to work with widgets and interactivity from one plot to another.  Unfortunately they do not work in the Google Jupyter environment...so instead I'll be showing you some things in Jupyter notebook on my own computer.  You are welcome to download this notebook yourself from: https://github.com/aruss175/transform_2020_intro_viz/tree/master/notebooks